In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix

In [ ]:
caminhos = [
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e4batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e4batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e4batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e4batch\5e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e8batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e8batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e8batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e8batch\5e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e16batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e16batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e16batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e16batch\5e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e32batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e32batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e32batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e32batch\5e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e64batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e64batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientDesativo\10e64batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientDesativo\10e64batch\5e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientAtivo\10e4batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientAtivo\10e4batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientAtivo\10e4batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientAtivo\10e4batch\5e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientAtivo\10e8batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientAtivo\10e8batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientAtivo\10e8batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientAtivo\10e8batch\5e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientAtivo\10e16batch\1e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientAtivo\10e16batch\1e-5\ModeloTreinado",
    },
    {
        "modelo": r"C:\Modelo\GradientAtivo\10e16batch\5e-5\modelo.pth",
        "tokenizador": r"C:\Modelo\GradientAtivo\10e16batch\5e-5\ModeloTreinado",
    },
]


In [ ]:
# Carregar o arquivo CSV
caminho_csv = r"C:\Modelo\test_data.csv"
data = pd.read_csv(caminho_csv, sep="|")

In [ ]:

previsoes = []
rotulos_reais = []


resultados_df = pd.DataFrame(columns=["Modelo", "Acurácia", "Precisão", "Recall", "F1-Score", "Kappa"])

total_modelos = len(caminhos)
modelo_atual = 1

In [ ]:

# Loop para testar cada modelo
for caminhos_modelo in caminhos:
    caminho_modelo = caminhos_modelo["modelo"]
    caminho_tokenizador = caminhos_modelo["tokenizador"]

    
    nome_modelo = caminho_modelo

    # Carregar o modelo treinado
    modelo_treinado = torch.load(caminho_modelo)

    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    modelo_treinado.to(device)

    # Inicializar o tokenizador
    tokenizer = AutoTokenizer.from_pretrained(caminho_tokenizador)

    progresso = 0
    # Preprocessar as frases e fazer as previsões uma por uma
    for _, row in data.iterrows():
        texto = row["text"]
        label = row["label"]

        # Tokenizar
        inputs = tokenizer(texto, return_tensors="pt", add_special_tokens=True, max_length=500, padding="max_length", truncation=True)
        inputs.to(device)  # Mover os dados de entrada para o mesmo dispositivo do modelo

        
        outputs = modelo_treinado(**inputs)
        probabilities = torch.softmax(outputs.logits, dim=1)

        # Determinar a classe prevista
        predicted_label = 1 if probabilities[0][1] > probabilities[0][0] else 0

        # Armazenar previsões e rótulos reais
        previsoes.append(predicted_label)
        rotulos_reais.append(label)
        progresso+= 1
        percent_concluido = (progresso / len(data)) * 100
        print(f"Progresso: {modelo_atual}/{total_modelos} modelos testados - {progresso}/{len(data)} ({percent_concluido:.2f}%)", end="\r")


    acuracia = accuracy_score(rotulos_reais, previsoes)
    precisao = precision_score(rotulos_reais, previsoes)
    recall = recall_score(rotulos_reais, previsoes)
    f1 = f1_score(rotulos_reais, previsoes)
    kappa = cohen_kappa_score(rotulos_reais, previsoes)

    # # Exibir métricas para o modelo atual
    # print(f"Modelo: {caminho_modelo}")
    # print(f"Acurácia: {acuracia:.2f}")
    # print(f"Precisão: {precisao:.2f}")
    # print(f"Recall: {recall:.2f}")
    # print(f"F1-Score: {f1:.2f}")
    # print(f"Kappa: {kappa:.2f}")

    # Adicionar resultados ao DataFrame
    resultados_df = pd.concat([resultados_df, pd.DataFrame({
    "Modelo": [nome_modelo],
    "Acurácia": [acuracia],
    "Precisão": [precisao],
    "Recall": [recall],
    "F1-Score": [f1],
    "Kappa": [kappa]
    })], ignore_index=True)

    # Limpar as listas para o próximo modelo
    previsoes = []
    rotulos_reais = []

    modelo_atual += 1


In [ ]:

# Salvar resultados em um arquivo CSV
resultados_csv = "modelos_avaliados.csv"
resultados_df.to_csv(resultados_csv, index=False)